In [1]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten,BatchNormalization
from keras.callbacks import EarlyStopping

from keras.constraints import maxnorm
from keras.utils import np_utils
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
seed = 16
filename = '../Datos/fer2013.csv'
label_map = ['Enfado', 'Asco', 'Miedo', 'Alegria', 'Tristeza', 'Sorpresa', 'Neutral']
names=['emocion','pixels','uso']
df=pd.read_csv(filename,names=names, na_filter=False)
im=df['pixels']
df.head(10)

,emocion,pixels,uso
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [3]:
def getData(filname):
    # Las imagenes 48x48 en grayscale (de 0 a 255)
    # N = 35887 (28709 de entrenamiento y 2304 de testeo)
    Y_train = []
    X_train = []
    Y_test = []
    X_test = []
    
    first = True
    uso = 'Training\n'
    for line in open(filname):
        #This condition skips the first condition
        if first:
            first = False
        else:
            row = line.split(',')
            condition = str(row[2])
            if condition == uso:
                Y_train.append(int(row[0]))
                X_train.append([int(p) for p in row[1].split()])
            else:
                Y_test.append(int(row[0]))
                X_test.append([int(p) for p in row[1].split()])
                
    X_train, Y_train = np.array(X_train)/255.0 , np.array(Y_train)
    X_test, Y_test = np.array(X_test)/255.0 , np.array(Y_test)

    return X_train, X_test, Y_train, Y_test 

In [4]:
X_train, X_test, Y_train, Y_test= getData(filename)

In [5]:
N_train = X_train.shape[0]
N_test = X_test.shape[0]
X_train = X_train.reshape(N_train, 48, 48,1)
X_test = X_test.reshape(N_test, 48, 48,1)

In [6]:
# one hot encode output
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)
class_num = Y_test.shape[1]

In [13]:
model = Sequential()

model.add(Conv2D(96,(11,11),input_shape=X_train.shape[1:],padding = 'same', strides = 1, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256,(5,5),input_shape=X_train.shape[1:],padding = 'same', strides = 1, activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(384,(3,3),input_shape=X_train.shape[1:],padding = 'same', strides = 1, activation = 'relu'))
model.add(Conv2D(384,(3,3),input_shape=X_train.shape[1:],padding = 'same', strides = 1, activation = 'relu'))
model.add(Conv2D(236,(3,3),input_shape=X_train.shape[1:],padding = 'same', strides = 1, activation = 'relu'))

model.add(Flatten())
model.add(Dense(4096, activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dense(4096, activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dense(class_num, activation='softmax',kernel_constraint=maxnorm(3)))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 48, 48, 96)        11712     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 24, 24, 96)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 24, 24, 256)       614656    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 12, 12, 384)       885120    
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 12, 12, 384)       1327488   
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 12, 12, 236)      

In [16]:
epochs = 100
optimizer = 'adam'
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.1, verbose = 1, patience = 10)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
np.random.seed(16)
cb =  [es]
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64, callbacks = cb)

Epoch 1/100
449/449 [==============================] - 2277s 5s/step - loss: 1.8902 - accuracy: 0.2516 - val_loss: 1.7892 - val_accuracy: 0.2517
Epoch 2/100
449/449 [==============================] - 2175s 5s/step - loss: 1.6587 - accuracy: 0.3375 - val_loss: 1.5572 - val_accuracy: 0.3876
Epoch 3/100
449/449 [==============================] - 2127s 5s/step - loss: 1.5108 - accuracy: 0.4092 - val_loss: 1.5054 - val_accuracy: 0.4048
Epoch 4/100
449/449 [==============================] - 2121s 5s/step - loss: 1.4016 - accuracy: 0.4587 - val_loss: 1.4017 - val_accuracy: 0.4492
Epoch 5/100
449/449 [==============================] - 2112s 5s/step - loss: 1.2735 - accuracy: 0.5142 - val_loss: 1.3867 - val_accuracy: 0.4706
Epoch 6/100
449/449 [==============================] - 2117s 5s/step - loss: 1.0988 - accuracy: 0.5871 - val_loss: 1.4016 - val_accuracy: 0.4837
Epoch 7/100
449/449 [==============================] - 2154s 5s/step - loss: 0.8548 - accuracy: 0.6819 - val_loss: 1.5524 - val_ac

In [8]:
model = Sequential()

model.add(Conv2D(96,(11,11),input_shape=X_train.shape[1:],padding = 'same', strides = 1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Conv2D(256,(5,5),input_shape=X_train.shape[1:],padding = 'same', strides = 1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Conv2D(384,(3,3),input_shape=X_train.shape[1:],padding = 'same', strides = 1))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Conv2D(384,(3,3),input_shape=X_train.shape[1:],padding = 'same', strides = 1))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Conv2D(236,(3,3),input_shape=X_train.shape[1:],padding = 'same', strides = 1))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(4096, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(class_num, kernel_constraint=maxnorm(3)))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 48, 48, 96)        11712     
_________________________________________________________________
activation_8 (Activation)    (None, 48, 48, 96)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 96)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 24, 24, 96)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 24, 24, 96)        384       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 256)       614656    
_________________________________________________________________
activation_9 (Activation)    (None, 24, 24, 256)      

In [9]:
epochs = 100
optimizer = 'adam'
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.1, verbose = 1, patience = 10)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
np.random.seed(16)
cb =  [es]
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=64, callbacks = cb)

Epoch 1/100
449/449 [==============================] - 2423s 5s/step - loss: 2.2557 - accuracy: 0.2546 - val_loss: 2.0182 - val_accuracy: 0.2888
Epoch 2/100
449/449 [==============================] - 57543s 128s/step - loss: 1.8626 - accuracy: 0.3257 - val_loss: 1.8756 - val_accuracy: 0.2689
Epoch 3/100
449/449 [==============================] - 2557s 6s/step - loss: 1.7680 - accuracy: 0.3679 - val_loss: 3.6769 - val_accuracy: 0.3544
Epoch 4/100
449/449 [==============================] - 2553s 6s/step - loss: 1.7047 - accuracy: 0.3948 - val_loss: 1.6426 - val_accuracy: 0.4245
Epoch 5/100
449/449 [==============================] - 2530s 6s/step - loss: 1.6494 - accuracy: 0.4183 - val_loss: 1.5247 - val_accuracy: 0.4432
Epoch 6/100
449/449 [==============================] - 2505s 6s/step - loss: 1.6223 - accuracy: 0.4302 - val_loss: 1.6379 - val_accuracy: 0.3805
Epoch 7/100
449/449 [==============================] - 2500s 6s/step - loss: 1.5959 - accuracy: 0.4441 - val_loss: 1.8690 - val